In [1]:
%pip install mayavi configobj pyyaml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import estimate, rospy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mayavi import mlab
from tvtk.api import tvtk
from datetime import datetime

from placo_utils.tf import tf as ptf
import math

%matplotlib qt

<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for std::vector<double, std::allocator<double> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignored.
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
/usr/local/lib/python3.8/dist-packages/traits/etsconfig/etsconfig.py:425: UserWarning: Environment variable "HOME" not set, setting home directory to /tmp
  warn(


In [4]:
load_from_file = True
reached_file = 'reached_10x10x10_2024_09_07-02_37_16_PM.sav'
rejected_file = 'rejected_10x10x10_2024_09_07-02_37_16_PM.sav'
shape = (10,)

In [5]:
if not load_from_file:
    rospy.init_node('test')
sampler = estimate.WorkspaceSampler(
    'base_footprint', '/kinematics_server/check_collision', '/kinematics_server/goal_pose', 
    shape, [-1.25, -0.45981], [-0.75, 0.75], [0.0, 2.1], prefix='ur_', connect=(not load_from_file)
)

In [6]:
if load_from_file:
    with open(f'log/{reached_file}', 'rb') as f:
        reached = np.load(f)
    
    with open(f'log/{rejected_file}', 'rb') as f:
        rejected = np.load(f)
else:
    reached, rejected = sampler.sample_reachable_points()

In [7]:
if not load_from_file:
    date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
    shape_str = f'{shape[0]}x{shape[1]}x{shape[2]}'
    with open(f'./log/reached_{shape_str}_{date}.sav', 'wb') as f:
        np.save(f, reached)
        
    with open(f'./log/rejected_{shape_str}_{date}.sav', 'wb') as f:
        np.save(f, rejected)

In [8]:
reached = np.array(reached)
rejected = np.array(rejected)
rxs, rys, rzs = reached[:, 0, 3], reached[:, 1, 3], reached[:, 2, 3]
nxs, nys, nzs = rejected[:, 0, 3], rejected[:, 1, 3], rejected[:, 2, 3]
pos = np.stack((rxs, rys, rzs), axis=1)

In [9]:
def cuboid_data(o, size=(1,1,1)):
    X = [[[0, 1, 0], [0, 0, 0], [1, 0, 0], [1, 1, 0]],
         [[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]],
         [[1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1]],
         [[0, 0, 1], [0, 0, 0], [0, 1, 0], [0, 1, 1]],
         [[0, 1, 0], [0, 1, 1], [1, 1, 1], [1, 1, 0]],
         [[0, 1, 1], [0, 0, 1], [1, 0, 1], [1, 1, 1]]]
    X = np.array(X).astype(float)
    for i in range(3):
        X[:,:,i] *= size[i]
    X += np.array(o)
    return X

def plotCubeAt(position, size=None, color='C0', **kwargs):
    cube = cuboid_data(position, size=size)
    return Poly3DCollection(cube, **kwargs)

In [10]:
def get_cuboid(pos, size=(1,1,1)):
    x, y, z = [0, 1], [0, 1], [0, 1]
    vertices = np.vstack(np.meshgrid(x,y,z)).reshape(3,-1).T
    vertices = np.array(vertices).astype(float)
    for i, s in enumerate(size):
        vertices[:, i] *= s
    vertices += np.array(pos)
    xx, yy, zz = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    faces = [
        [0, 1, 2],
        [1, 2, 3],
        [0, 1, 4],
        [1, 5, 4],
        [1, 3, 5],
        [3, 7, 5],
        [2, 3, 6],
        [3, 7, 6],
        [0, 2, 4],
        [2, 6, 4],
        [4, 5, 6],
        [5, 7, 6]
    ]
    return xx, yy, zz, faces

def plot_cuboid(pos, size=(1,1,1), **kwargs):
    xx, yy, zz, faces = get_cuboid(pos, size)
    return mlab.triangular_mesh(xx, yy, zz, faces, **kwargs)

In [11]:
def isin_tolerance(A, B, tol):
    A = np.asarray(A)
    B = np.asarray(B)

    Bs = np.sort(B) # skip if already sorted
    idx = np.searchsorted(Bs, A)

    linvalid_mask = idx==len(B)
    idx[linvalid_mask] = len(B)-1
    lval = Bs[idx] - A
    lval[linvalid_mask] *=-1

    rinvalid_mask = idx==0
    idx1 = idx-1
    idx1[rinvalid_mask] = 0
    rval = A - Bs[idx1]
    rval[rinvalid_mask] *=-1
    return np.minimum(lval, rval) <= tol

In [12]:
class VoxelPlotter:
    def __init__(self, fig, sampler, pos, centroids) -> None:
        
        self.index = -1
        self.fig = fig
        self.fig.scene.interactor.add_observer('KeyPressEvent', self.handle_keypress)
        
        self.pos = pos
        self.Z = sampler.zs
        self.n_z = len(self.Z)
        
        self.xstep = sampler.xs[1] - sampler.xs[0]
        self.ystep = sampler.ys[1] - sampler.ys[0]
        self.zstep = sampler.zs[1] - sampler.zs[0]

        xmin = np.min(sampler.xs) - self.xstep
        xmax = np.max(sampler.xs) + self.xstep
        ymin = np.min(sampler.ys) - 1
        ymax = np.max(sampler.ys) + 1
        zmin = np.min(sampler.zs) - self.zstep
        zmax = np.max(sampler.zs) + self.zstep
        self.bounds = np.array([xmin, xmax, ymin, ymax, zmin, zmax])
        
        # Define mlab pipeline
        source = mlab.pipeline.scalar_scatter(pos[:, 0], pos[:, 1], pos[:, 2], figure=fig)
        source2 = mlab.pipeline.scalar_scatter(centroids[:, :, 0], centroids[:, :, 1], centroids[:, :, 2], figure=fig)

        
        self.clip = mlab.pipeline.data_set_clipper(source, figure=fig)
        self.clip.widget.widget.enabled = False
        
        self.clip.widget.widget_mode = 'Box'
        self.clip.filter.clip_function.set_bounds(self.bounds)
        self.clip.filter.inside_out = True
        self.clip.filter.use_value_as_offset = False

        
        self.clip_reach = mlab.pipeline.data_set_clipper(source2, figure=fig)
        self.clip_reach.widget.widget.enabled = False
        
        self.clip_reach.widget.widget_mode = 'Box'
        self.clip_reach.filter.clip_function.set_bounds(self.bounds)
        self.clip_reach.filter.inside_out = True
        self.clip_reach.filter.use_value_as_offset = False
        
        
        self.points = mlab.pipeline.glyph(self.clip, figure=fig, color=(0.5, 0., 0.5), mode='cube')
        self.points.glyph.glyph_source.glyph_source.x_length = self.xstep * 0.9
        self.points.glyph.glyph_source.glyph_source.y_length = self.ystep * 0.9
        self.points.glyph.glyph_source.glyph_source.z_length = self.zstep * 0.9

        self.reachable_points = mlab.pipeline.glyph(self.clip_reach, figure=fig, color=(1.0, 0.65, 0.), mode='sphere')
        self.reachable_points.glyph.glyph_source.glyph_source.radius = self.xstep * 0.9
        self.reachable_points.visible = False
        
        
        self.z_label = mlab.text(0.95, 0.9, '', figure=fig, width=0.10)
        self.z_label.visible = False
        self.z_label.actor.text_scale_mode = 'viewport'
        self.z_label.property.background_color = (170/255, 170/255, 255/255)
        self.z_label.property.background_opacity = 0.5
        self.z_label.property.justification = 'right'
        self.z_label.property.vertical_justification = 'centered'
    
        mlab.draw(figure=self.fig)
    
    
    def handle_keypress(self, vtk_obj, event) -> None:
        key = vtk_obj.GetKeySym()
        if key == 'Home':
            self.index = (self.index + 1) if self.index < self.n_z - 1 else 0
            self.update()
        elif key == 'End':
            self.index = (self.index - 1) if self.index > 0 else self.n_z - 1
            self.update()
        elif key == 'BackSpace':
            self.index = -1
            self.update()
        
    
    def update(self) -> None:
        if self.index < 0:
            self.z_label.text = ''
            self.clip.filter.clip_function.set_bounds(self.bounds)
            self.reachable_points.visible = False
        else:
            self.reachable_points.visible = True
            self.z_label.visible = True
            self.z_label.text = f'Current Z : {self.Z[self.index]:.5f}'
            zmin = self.Z[self.index] - self.zstep / 2
            zmax = self.Z[self.index] + self.zstep / 2
            self.clip.filter.clip_function.set_bounds([*self.bounds[:4], zmin, zmax])
            self.clip_reach.filter.clip_function.set_bounds([*self.bounds[:4], zmin, zmax])
        
        self.z_label.update_pipeline()
        mlab.draw(figure=self.fig)

In [13]:
def get_angles(pos) -> list:
        shifted = pos.copy()
        shifted[:, 0] -= -0.45981# max X in setup, should probably parametrize but oh well
        # shifted[:, :2] *= -1
        shifted[:, 0] *= -1
        angles = np.arctan2(shifted[:, 0], shifted[:, 1])
        
        return list(zip(pos, angles))


def find_centroids(pos, Z, n_sections : int = 5):
    points, angles = list(zip(*get_angles(pos)))
    points, angles = np.array(points), np.array(angles)
    centroids = [[] for _ in range(Z.shape[0])]
    sections = []
    previous = 0
    angle_step = np.pi / n_sections
    for i in range(n_sections):
        current = angle_step * (i + 1)
        sections.append(points[(angles < current) & (angles >= previous)])
        previous = current
        
    sections = np.array(sections)
    for i, z in enumerate(Z):
        for section in sections:
            P = section[np.isclose(section[:, 2], z)]
            if not len(P):
                centroids[i].append([float('nan')] * 3)
                continue
            x = P[:, 0].mean()
            y = P[:, 1].mean()
            centroids[i].append([x, y, z])
            
    return np.array(centroids)

In [14]:
reachable_centroids = find_centroids(pos, sampler.zs)

In [15]:
config = {'layers' : []}
for i, centroids in enumerate(reachable_centroids):
    if np.isnan(centroids).all():
        continue
    config['layers'].append({
        'z' : float(sampler.zs[i]),
        'points' : [{'x': x, 'y': y} for x, y in zip(centroids[:, 0].tolist(), centroids[:, 1].tolist())],
    })

In [16]:
import yaml, json
with open(f'../config/reachable_points.yaml', 'w') as f:
    yaml.dump(config, f)
with open(f'../config/reachable_points.json', 'w') as f:
    json.dump(config, f)

In [ ]:
visible = True
def handle_keypress(event, _):
    global visible, points
    key = event.GetKeySym()
    if key == 'x':
        visible = not visible
        points.visible = visible

fig = mlab.figure(bgcolor=(1., 1., 1.),fgcolor=(0., 0., 0.))
points = mlab.points3d(nxs, nys, nzs, figure=fig, color=(1., 0., 0.), opacity=0.25, scale_factor=0.025)
mlab.axes()

t = VoxelPlotter(fig, sampler, pos, reachable_centroids)
rr100 = plot_cuboid([-0.860/2, -0.65850 / 2, 0.0], size=(0.45981 * 2, 0.65850, 0.80091), figure=fig, color=(0.,0.,1.), opacity=0.5)

fig.scene.interactor.add_observer('KeyPressEvent', handle_keypress)

mlab.show()

libpng warning: iCCP: known incorrect sRGB profile


In [ ]:
manipulable, n_manipulable = sampler.sample_manipulable_points(reached)

In [ ]:
manipulable = np.array(manipulable)
n_manipulable = np.array(n_manipulable)
xs, ys, zs = manipulable[:, 0, 3], manipulable[:, 1, 3], manipulable[:, 2, 3]
nxs, nys, nzs = n_manipulable[:, 0, 3], n_manipulable[:, 1, 3], n_manipulable[:, 2, 3]
pos = np.stack((xs, ys, zs), axis=1)

In [ ]:
visible = True
def handle_keypress(event, _):
    global visible, points
    key = event.GetKeySym()
    if key == 'x':
        visible = not visible
        points.visible = visible

fig = mlab.figure(bgcolor=(1., 1., 1.),fgcolor=(0., 0., 0.))
fig.scene.interactor.add_observer('KeyPressEvent', handle_keypress)
        
points = mlab.points3d(nxs, nys, nzs, figure=fig, color=(1., 0., 0.), opacity=0.25, scale_factor=0.025)
mlab.axes()

t = VoxelPlotter(fig, sampler, pos)
rr100 = plot_cuboid([-0.860/2, -0.65850 / 2, 0.0], size=(0.45981 * 2, 0.65850, 0.80091), figure=fig, color=(0.,0.,1.), opacity=0.5)

mlab.show()